**Импорт библиотек**

In [1]:
!pip install transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [3]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=f721731dc67a07162047a2c243b7914bed1ffd1ba60a9eb08e45afcd3819cccf
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np
from datasets import Dataset as HFDataset
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
import evaluate
import os
os.environ["WANDB_DISABLED"] = "true"

In [5]:
file_path = "/content/drive/MyDrive/course_project/processed_data_all.csv"
df = pd.read_csv(file_path)

In [6]:
model_name = "cointegrated/rubert-tiny2"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
all_labels = set()
for entities in df["entities"]:
    for entity in eval(entities):
        all_labels.add(entity["label"])

label_list = ["O"] + sorted(all_labels)
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

print("Обнаруженные метки:", label_list)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Обнаруженные метки: ['O', 'B-DATE', 'B-LAW', 'B-MONEY', 'B-PER', 'B-REGION']


In [7]:
def tokenize_and_align_labels(example):
    tokens = tokenizer(text=example["text"], padding="max_length", truncation=True, max_length=512, return_offsets_mapping=True)
    offset_mapping = tokens.pop("offset_mapping")

    labels = ["O"] * len(tokens["input_ids"])

    for entity in eval(example["entities"]):
        start, end, entity_label = entity["start"], entity["end"], entity["label"]
        for idx, (token_start, token_end) in enumerate(offset_mapping):
            if token_end > start and token_start < end:
                labels[idx] = entity_label

    tokens["labels"] = [label2id[label] for label in labels]
    return tokens

In [8]:
train_texts, val_texts = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

train_dataset = HFDataset.from_pandas(train_texts)
val_dataset = HFDataset.from_pandas(val_texts)

train_dataset = train_dataset.map(tokenize_and_align_labels, batched=False)
val_dataset = val_dataset.map(tokenize_and_align_labels, batched=False)


Map:   0%|          | 0/9527 [00:00<?, ? examples/s]

Map:   0%|          | 0/2382 [00:00<?, ? examples/s]

In [9]:
model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), id2label=id2label, label2id=label2id)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    weight_decay=0.01,
    save_total_limit=2,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [11]:
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[id2label[label] for label in label_seq if label != -100] for label_seq in labels]
    true_predictions = [[id2label[pred] for pred, label in zip(pred_seq, label_seq) if label != -100] for pred_seq, label_seq in zip(predictions, labels)]

    return metric.compute(predictions=true_predictions, references=true_labels)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Date,Law,Money,Per,Region,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.020400,0.016224,"{'precision': 0.992343413123998, 'recall': 0.9978320687067456, 'f1': 0.9950801724018459, 'number': 35979}","{'precision': 0.990722385219253, 'recall': 0.9995999679974398, 'f1': 0.9951413779370769, 'number': 49996}","{'precision': 0.9635687732342008, 'recall': 0.9880559085133418, 'f1': 0.9756587202007527, 'number': 3935}","{'precision': 0.9654071551164507, 'recall': 0.9986477536151893, 'f1': 0.9817461645241659, 'number': 108708}","{'precision': 0.9783544877606528, 'recall': 0.9938982270320055, 'f1': 0.986065105653912, 'number': 8686}",0.976535,0.998336,0.987315,0.995655
2,0.011000,0.012926,"{'precision': 0.994492721536503, 'recall': 0.998777064398677, 'f1': 0.9966302885748756, 'number': 35979}","{'precision': 0.9973452034013334, 'recall': 0.9993799503960317, 'f1': 0.9983615401522569, 'number': 49996}","{'precision': 0.9726300074645434, 'recall': 0.9933926302414231, 'f1': 0.98290168468695, 'number': 3935}","{'precision': 0.9692865046221937, 'recall': 0.9992456856901056, 'f1': 0.9840381200855165, 'number': 108708}","{'precision': 0.9834128607134742, 'recall': 0.9965461662445314, 'f1': 0.989935956084172, 'number': 8686}",0.980911,0.998973,0.989860,0.996532
3,0.008700,0.011490,"{'precision': 0.9950721187121065, 'recall': 0.9989994163261903, 'f1': 0.9970319001386964, 'number': 35979}","{'precision': 0.9974453647340584, 'recall': 0.9996199695975678, 'f1': 0.9985314832019659, 'number': 49996}","{'precision': 0.9807307307307307, 'recall': 0.9959339263024143, 'f1': 0.9882738620602698, 'number': 3935}","{'precision': 0.9725435727407142, 'recall': 0.9983717849652279, 'f1': 0.9852884436435274, 'number': 108708}","{'precision': 0.9880205362236166, 'recall': 0.9970066774119273, 'f1': 0.9924932668614979, 'number': 8686}",0.983132,0.998678,0.990844,0.996873
4,0.010500,0.010486,"{'precision': 0.9959543338505874, 'recall': 0.9989716223352512, 'f1': 0.9974606962965019, 'number': 35979}","{'precision': 0.9978832175093857, 'recall': 0.9994799583966717, 'f1': 0.9986809497162044, 'number': 49996}","{'precision': 0.9790575916230366, 'recall': 0.9979669631512071, 'f1': 0.9884218474704254, 'number': 3935}","{'precision': 0.9749037873610762, 'recall': 0.9973598999153697, 'f1': 0.9860040014550746, 'number': 108708}","{'precision': 0.9897272000913138, 'recall': 0.9982730831222657, 'f1': 0.9939817733707801, 'number': 8686}",0.984692,0.998201,0.991400,0.997062
5,0.005400,0.013179,"{'precision': 0.9957891237498961, 'recall': 0.9990550043080686, 'f1': 0.99741939064321, 'number': 35979}","{'precision': 0.9978041720730612, 'recall': 0.9997799823985919, 'f1': 0.99879110009891, 'number': 49996}","{'precision': 0.9815092453773113, 'recall': 0.9982210927573062, 'f1': 0.9897946327327705, 'number': 3935}","{'precision': 0.9734178463524623, 'recall': 0.99878573794017, 'f1': 0.9859386421731766, 'number': 108708}","{'precision': 0.9909724602902525, 'recall': 0.9983882109141147, 'f1': 0.9946665137351609, 'number': 8686}",0.983942,0.999045,0.991436,0.997074
6,0.004700,0.011693,"{'precision': 0.9965900584957446, 'recall': 0.999138386280886, 'f1': 0.9978625954198473, 'number': 35979}","{'precision': 0.9983422876430526, 'recall': 0.9997999839987199, 'f1': 0.9990706041073303, 'number': 49996}","{'precision': 0.9886678418534374, 'recall': 0.997712833545108, 'f1': 0.9931697444978497, 'number': 3935}","{'precision': 0.9771347103038178, 'recall': 0.9961456378555396, 'f1': 0.9865485967813202, 'number': 108708}","{'precision': 0.9928964252978918, 'recall': 0.997697444163021, 'f1': 0.9952911450557023, 'number': 8686}",0.986421,0.997641,0.991999,0.997271
7,0.003800,0.013451,"{'precision': 0.9972255361651361, 'recall': 0.9989994163261903, 'f1': 0.9981116880953042, 'number': 35979}","{'precision': 0.9988408345990887, 'recall': 0.9996399711976958, 'f1': 0.9992402431222009, 'number': 49996}","{'precision': 0.9876822523881348, 'recall': 0.998475222

Trainer is attempting to log a value of "{'precision': 0.992343413123998, 'recall': 0.9978320687067456, 'f1': 0.9950801724018459, 'number': 35979}" of type <class 'dict'> for key "eval/DATE" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.990722385219253, 'recall': 0.9995999679974398, 'f1': 0.9951413779370769, 'number': 49996}" of type <class 'dict'> for key "eval/LAW" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9635687732342008, 'recall': 0.9880559085133418, 'f1': 0.9756587202007527, 'number': 3935}" of type <class 'dict'> for key "eval/MONEY" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9654071551164507, 'recall': 0.9986477536151

TrainOutput(global_step=17865, training_loss=0.0079120701100451, metrics={'train_runtime': 1633.936, 'train_samples_per_second': 87.461, 'train_steps_per_second': 10.934, 'total_flos': 1011489290864640.0, 'train_loss': 0.0079120701100451, 'epoch': 15.0})

In [14]:
trainer.evaluate()

Trainer is attempting to log a value of "{'precision': 0.9977239292752658, 'recall': 0.9990550043080686, 'f1': 0.9983890231369608, 'number': 35979}" of type <class 'dict'> for key "eval/DATE" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9990604322011875, 'recall': 0.9995999679974398, 'f1': 0.9993301272758176, 'number': 49996}" of type <class 'dict'> for key "eval/LAW" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9909205548549811, 'recall': 0.9984752223634054, 'f1': 0.9946835443037974, 'number': 3935}" of type <class 'dict'> for key "eval/MONEY" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9775975314879642, 'recall': 0.99673437097

{'eval_loss': 0.01976815238595009,
 'eval_DATE': {'precision': 0.9977239292752658,
  'recall': 0.9990550043080686,
  'f1': 0.9983890231369608,
  'number': 35979},
 'eval_LAW': {'precision': 0.9990604322011875,
  'recall': 0.9995999679974398,
  'f1': 0.9993301272758176,
  'number': 49996},
 'eval_MONEY': {'precision': 0.9909205548549811,
  'recall': 0.9984752223634054,
  'f1': 0.9946835443037974,
  'number': 3935},
 'eval_PER': {'precision': 0.9775975314879642,
  'recall': 0.9967343709754571,
  'f1': 0.987073206282112,
  'number': 108708},
 'eval_REGION': {'precision': 0.9942706542912799,
  'recall': 0.9989638498733594,
  'f1': 0.9966117268707287,
  'number': 8686},
 'eval_overall_precision': 0.9871265113704683,
 'eval_overall_recall': 0.9979546945548566,
 'eval_overall_f1': 0.9925110702788799,
 'eval_overall_accuracy': 0.9974442104848866,
 'eval_runtime': 22.5239,
 'eval_samples_per_second': 105.754,
 'eval_steps_per_second': 13.23,
 'epoch': 15.0}

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
trainer.save_model("/content/drive/MyDrive/truth_of_the_matter_model_rubert_tiny2")
tokenizer.save_pretrained("/content/drive/MyDrive/truth_of_the_matter_model_rubert_tiny2")

('/content/drive/MyDrive/truth_of_the_matter_model_rubert_tiny2/tokenizer_config.json',
 '/content/drive/MyDrive/truth_of_the_matter_model_rubert_tiny2/special_tokens_map.json',
 '/content/drive/MyDrive/truth_of_the_matter_model_rubert_tiny2/vocab.txt',
 '/content/drive/MyDrive/truth_of_the_matter_model_rubert_tiny2/added_tokens.json',
 '/content/drive/MyDrive/truth_of_the_matter_model_rubert_tiny2/tokenizer.json')